In [15]:
"""

1. get the data with np category as label and additional comments as x(iput)
2. clean the text and split the data in to training and test
3. 

"""
import re
import nltk
import spacy
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

nlp = spacy.load("en_core_web_sm")
nltk.download('stopwords')
SENTIMENT_FILE = "Verbatim comments_14Mar19.xlsx"
punctuations = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~©'


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bhaskark847\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Install Bert server and client#
    !pip install bert-serving-server
    !pip install bert-serving-client 

In [16]:
def cleanup_text(docs):
    """
        Tokenize the data and remove the stop words . 
    """    
    texts = []
    sentence_list=[]    
    docs =   docs.replace("\n", "").replace('\t','') # We dont need to understand the line breaks.Thus removing     
    doc = nlp(docs)
    for idx, sentence in enumerate(doc.sents): # we are using spacy for sentence tokenization
        sentence_list.append(re.sub(r'[^\w\s]','',str(sentence)))
    
    tokens = [tok for tok in sentence_list if tok not in stopwords.words("english") and tok not in punctuations]
    return ''.join(tokens)

In [17]:
tempDF.to_pickle("data.pkl")

In [18]:
sentiment_df = pd.read_excel(SENTIMENT_FILE)
x = sentiment_df["Additional Comments"].apply(cleanup_text)
y =  sentiment_df["NP Category"].astype('category').cat.codes

pd.DataFrame({'input':x,'label':y})
X_tr, X_val, y_tr, y_val = train_test_split(x,y,test_size=0.25, random_state=42)

In [19]:
from bert_serving.client import BertClient

# make a connection with the BERT server using it's ip address
bc = BertClient(ip="localhost",check_length=False)
# get the embedding for train and val sets
X_tr_bert = bc.encode(X_tr.tolist())
X_val_bert = bc.encode(X_val.tolist())

# np.savetxt('x_training.txt', X_tr_bert)
np.savetxt('x_validation.txt', X_val_bert)

In [49]:

from sklearn.linear_model import LogisticRegression

# LR model
model_bert = LogisticRegression()
# train
model_bert = model_bert.fit(X_tr_bert, y_tr)
# predict
pred_bert = model_bert.predict(X_val_bert)

c:\users\bhaskark847\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [50]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_val, pred_bert))

0.9036144578313253


In [161]:
# sentiment = 'Thank you'
from sklearn.cluster import KMeans

num_clusters = 3
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(X_tr_bert)
cluster_assignment = clustering_model.labels_


df_temp = {'Negative': x.iloc[np.where(cluster_assignment == 2 )].values
           ,'Neutral':x.iloc[np.where(cluster_assignment == 0 )].values
           ,'Positive':x.iloc[np.where(cluster_assignment == 1 )].values}
df_temp = pd.DataFrame.from_dict(df_temp,orient='index')
df_temp.transpose().head(25)

,Negative,Neutral,Positive
0,Scale up the quality of customization commit...,Can spend more time understanding our business...,1RACI matrix should be discussed and finalized...
1,Adherence to quality of work and timelines nee...,Deeper insights into the deal and stepping bac...,a People are found to be good in terms of atti...
2,Basis on feedback provided PWC has picked up c...,Initial level PWC involvement was fantastic bu...,appreciate the efforts and the knowledge share...
3,Business need swift responses to take decision...,My Experience in the GST roll out was very bad...,Are we only responding socio political economy...
4,Develop and retain talent rather than acquire ...,Need to strengthen its expertise,Aucune rapidité nos enjeux étaient urgents etn...
5,Planning and allocating resour,PwC needs to first complete the Concurrent Aud...,Better mechanism and more frequent governance
6,Preferential pricing for long standing clients...,PwC team needs to be more accommodative in lis...,Current issues with communication with the PwC...
7,Proactive in sharing new ideasthoughts in orde...,Quite disappointed in my last couple of engage...,Given PWCs expertise on the taxation side the ...
8,proactive suggestions during initial assessments,The PWC Team has been working in right earnest...,I believe the team that I work with is respons...
9,Really needs to be proactive in coming up with...,The ratings are provided for All PWC services ...,I think Your services are too expensive and th...


In [163]:
df_temp.transpose()

,Negative,Neutral,Positive
0,Scale up the quality of customization commit...,Can spend more time understanding our business...,1RACI matrix should be discussed and finalized...
1,Adherence to quality of work and timelines nee...,Deeper insights into the deal and stepping bac...,a People are found to be good in terms of atti...
2,Basis on feedback provided PWC has picked up c...,Initial level PWC involvement was fantastic bu...,appreciate the efforts and the knowledge share...
3,Business need swift responses to take decision...,My Experience in the GST roll out was very bad...,Are we only responding socio political economy...
4,Develop and retain talent rather than acquire ...,Need to strengthen its expertise,Aucune rapidité nos enjeux étaient urgents etn...
...,...,...,...
199,None,None,PwC Transfer Pricing colleagues are very suppo...
200,None,None,PwC was available when required for discussion
201,None,None,Rajan Shankar and their team did an excellent ...
202,None,None,Really very good team


In [143]:
 sentiment_df["NP Category"].astype('category')

0      Detractor
1      Detractor
2      Detractor
3      Detractor
4      Detractor
         ...    
658     Promoter
659     Promoter
660     Promoter
661     Promoter
662     Promoter
Name: NP Category, Length: 663, dtype: category
Categories (2, object): [Detractor, Promoter]

In [95]:
# build_model(X_tr_bert,y_tr,None)